# Numeric to word

Converts:
- General number and decimals (Distance, weight, etc)
- Time
    - Hour (enhanced from https://sukhbinder.wordpress.com/2013/12/29/time-in-words-with-python/)
    - Date (format is following from https://en.wikipedia.org/wiki/Date_format_by_country)
    - Year
- Phone number (https://stackabuse.com/validating-and-formatting-phone-numbers-in-python)
- Money (the currency symbols)
    - Find the currency symbols in text:
        - https://www.geeksforgeeks.org/how-to-find-index-of-any-currency-symbols-in-a-given-string/
    - The symbols:
        - https://thefactfile.org/countries-currencies-symbols/ (taken)
        - https://wise.com/gb/blog/world-currency-symbols
- Sticky number:
    - 2x, 5th, 8pm, 10%
    - 10-5 mol: if mol then 10^-5, then ten power minus five
    - 5-14 years: if year then five to fourteen years
- Math:
    - Equal: '='
    
Unhandled:
- N2, N2O

TO DO:
- Corner cases check (please write your test cases in this notebook)
- Implementation to NL-augmenter

Further improvements:
- Word to numeric
    - https://pypi.org/project/word2number-i18n/
    - https://pypi.org/project/word2number/
- All lang numeric
    - https://pypi.org/project/fastlangid/

In [1]:
import json 
import numeric2word

#### Implementation

In [2]:
# TO DO: Check whether all the desired capabilities are met

transformed_sentences = []
for sentence in [    
#                 "Please buy me 20 apples", #done
#                 "Please buy me 20000 apples", #done
#                 "The accuracy is 99.22", #done
#                 "The size is 12,65 L", #done
#                 "The size is 200,000,000.65 L", #done
# #                 "The size is (12,235)", #done
#                 # Date
#                 "The deadline is in 2020/01/02", #done
#                 "The deadline is in 2020/01", #unhandled
#                 "The deadline is in Jan 2020", #done
#                 "This is 2020!", #done
#                 # Time
#                 "Slow down, it\'s still 5.05", #unhandled, considered as a general number in default
#                 "Slow down, it\'s still 5:10", #done
#                 "Slow down, it\'s still 5:29", #done
#                 "Quick!, it\'s already 23:00", #done
#                 "Quick!, it\'s already 11:30", #done
#                 "Quick!, it\'s already 9:45", #done
#                 "Quick!, it\'s already 1:50", #done
#                 # Date Time
#                 "Set up a schedule for Monday, June 5th 2021 at 10:00", #done
#                 "Your christmas dinner is on 25 December 2021 at 8pm", #done
#                 "You have an appointment on 14/07/2021 at 6pm", #done
#                 "You have an appointment on 07/14/2021 at 6pm", #done
#                 "You have an appointment on 07/07/1991 at 6pm", #done
#                 # Percentage
#                 "He only use 20% of my power", #done
#                 "My phone's battery is less than 10%", #done
#                 # Phone Number
#                 "Call 911 now!", #done
#                 "Check your credit by dialing *123#", #done
#                 "My phone number is +1371893178", #done
#                 "My phone number is +6287822216501", #done
#                 "My phone number is 6287822", #done
#                 "My phone number is 628782", #done
#                 "My phone number is 0741-71893178", #done
#                 "My phone number is (+62878)-222-165-01", #done
#                 # Currency
#                 "The price is $.2", #done
#                 "The price is .2USD", #done
#                 "The price is 2¢", #done
#                 "The price is 300.2$", #done
#                 "The price is USD300.2", #done
#                 "The price is 300.2USD", #done
#                 "The price is USD300!@#!", #unhandled
#                 "The size is (\$12,235)", #unhandled
#                 "The size is ($12,235)", #done 
#                 # SemEval 2019 Task 10: Math Question Answering (https://www.aclweb.org/anthology/S19-2153.pdf)
#                 "Suppose 3x + y = 15, where x is a positive integer. What is the difference between the largest possible value of y and the smallest possible value of x, assuming that y is also a positive integer?", #done
#                 "At a basketball tournament involving 8 teams, each team played 4 games with each of the other teams. How many games were played at this tournament?", #done
#                 "The lengths of two sides of a triangle are (x − 2) and (x + 2), where x > 2. Which of the following ranges includes all and only the possible values of the third side y? (A) 0 < y < x (B) 0 < y < 2x (C) 4 < y < 2x", #done
#                 # ChemistryQA (https://openreview.net/pdf?id=oeHTRAehiFF)
#                 "At a particular temperature a 2.00 L flask at equilibrium contains 2.80 × 10−4 mol N2, 2.50 × 10−5 mol O2, and 2.00 × 10−2 mol N2O. How would you calculate K at this temperature for the following reaction: N2(g) + O2(g) → N2O(g) ?", #done
#                 "Volume of the flask is 2.00L. provided Mole of N2 is 2.80 × 10−4 mol. Mole of O2 is 2.50 × 10−5 mol. Mole of N2O is 2.00 × 10−2 mol. Reaction equation is N2(g) + O2(g) → N2O(g).", #done
#                 "How many moles of ammonium nitrate are in 335 mL of 0.425 M NH4NO3?", #done unhandled
#                 "What is the empirical formula of magnesium chloride if 0.96 g of magnesium combines with 2.84 g of chlorine?", #done
#                 "How would you write a balanced equation for the combustion of octane, C8H18 with oxygen to obtain carbon dioxide and water?", #done unhandled
#                 # PubmedQA (https://www.aclweb.org/anthology/D19-1259.pdf)
#                 "221 patients underwent CABG in our hospital from 2004 to 2007. 14 patients with preoperative AF and 4 patients with concomitant valve surgery", #done
#                 "The overall incidence of postoperative AF was 26%. Postoperative AF was significantly lower in the Statin group compared with the Non-statin group (16% versus 33%, p=0.005).", #done
#                 "30-day mortality was 12.4% in those aged<70 years and 22% in those>70 years (p<0.001).", #done
#                 "A total of 4 children aged 5-14 years with a sternal fracture were treated in 2 years, 2 children were hospitalized",
#                 # SMD / KVRET (https://www.aclweb.org/anthology/2020.findings-emnlp.215/)
#                 "Ok setting your medicine appointment for 7pm", #done
#                 "It will be between 20-30F in Alhambra on Friday.",
#                 "Safeway is located at 452 Arcadia Pl.", #done
#                 "The closest parking garage is Civic Center Garage, located 4 miles away at 270 Altaire Walk.", #done
#                 "Valero is at 200 Alester Avenue.", #done
#                 "Setting reminder for yoga activity with your mother on the 15th at 3pm ", #done
#                 "You will find Pizza Chicago at 915 Arbol Dr. It is 2 miles away without any traffic", #done
# #                 Mathematics Dataset (https://openreview.net/pdf?id=H1gR5iR5FX)
#                 "Solve -42*r + 27*c = -1167 and 130*r + 4*c = 372 for r.", #done
#                 "Calculate -841880142.544 + 411127.", #done
#                 "Let x(g) = 9*g + 1. Let q(c) = 2*c + 1. Let f(i) = 3*i - 39. Let w(j) = q(x(j)). Calculate f(w(a)).", #done 
#                 "Let e(l) = l - 6. Is 2 a factor of both e(9) and 2?", #done
#                 "Let u(n) = -n**3 - n**2. Let e(c) = -2*c**3 + c. Let l(j) = -118*e(j) + 54*u(j). What is the derivative of l(a)?", #done 
# #                 PubMed 200k RCT (https://www.aclweb.org/anthology/I17-2052.pdf)
#                 "A total of 24 male patients with chronic AT (median disease duration, 33 months) were randomized (1:1) to receive either a blinded injection of PRP (n = 12) or saline (n = 12). Patients were informed that they could drop out after 3 months if they were dissatisfied with the treatment.", #done 
#                 "Secondary outcomes were pain at rest (MD, 1.6; 95% CI, -0.5 to 3.7; P = .137), pain while walking (MD, 0.8; 95% CI, -1.8 to 3.3; P = .544), pain when tendon was squeezed (MD, 0.3; 95% CI, -0.2 to 0.9; P = .208).", #done
#                 "RESULTS	Planned contrasts indicated that CB group resulted in lower depressive symptom severity than brochure control at posttest ( p = .03 , d = 0.29 ) but not 6-month follow-up", #done 
#                 "METHODS	Twelve young ( 18-40 years ) and 12 elderly ( > 65 years and < / = 85 years ) subjects received a single 800-mg oral dose of telithromycin or an intravenous infusion of 400 mg ( young subjects ) or 480 mg ( elderly subjects ) of telithromycin over 2.5 h in two treatment periods , separated by a 1-week washout period .", #done 
#                 "RESULTS	At week 26 , HbA1c was significantly reduced from baseline with canagliflozin 100 and 300mg compared with placebo ( -0.77 , -1.03 and 0.14 % , respectively ; p < 0.001 for both ) .", #done
                "BACKGROUND	Australia New Zealand Clinical Trials Registry : ACTRN12613000290796 Registered 14 March 2013 .",
                "RESULTS	Significant improvements in glucose were observed with repaglinide ( HBA ( 1c ) : -1.5 % , fasting glucose : -2.8 mmol/L , 2-h glucose : -3.7 mmol/L , AUC ( 120 ) : -18.9 % ) and glibenclamide ( -1.0 % , -2.2 mmol/L , -2.5 mmol/L , -17.5 % ) .",
                "RESULTS	Repaglinide was also associated with an increase in the AUC ( 60 ) and AUC ( 120 ) for insulin ( +56 % , +61 % ) and C-peptide ( +41 % , +36 % ) .",
#                 BBC News (https://www.kaggle.com/c/learn-ai-bbc)
                "1833,worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.", #done 
                "last week  mci agreed to a buyout by verizon communications in a deal valued at $6.75bn.,business",
                "the sequel took $44.7m (£23.2m) between 24 and 26 december  according to studio estimates",
                "australian open champion williams survived an early scare to beat russia s elena bovina 1-6 6-1 6-4. world number one lindsay davenport and anastasia myskina also progressed",
                "odds on kenzie s victory have been slashed from 4/6 to 1/2  although bookmakers have said they are not ruling out a late rally from former happy mondays star bez",
                "other pre-electronic gadgets in the top 100 include the sextant from 1731 (59th position)  the marine chronometer from 1761 (42nd position) and the kodak brownie camera from 1900 (28th position)",
                "moya fights back for indian title carlos moya became the first man to successfully defend the chennai open title by beating four-times finalist paradorn srichaphan 3-6 6-4 7-6 (7/5)",
#                 CNN/Daily News (https://huggingface.co/datasets/cnn_dailymail)
            ]:
    transformed_words = []
    for i, word in enumerate(sentence.split()):
        prev_word = sentence.split()[i-1] if i > 0 else ' ' # beginning of sentence
        next_word = sentence.split()[i+1] if i < len(sentence.split())-1 else ' ' # end of sentence
#         print(word, next_word)
        transformed_word = numeric2word.recognize_transform(word, prev_word, next_word)
        transformed_words.append(transformed_word)
    transformed_sentence = ' '.join(transformed_words)
    transformed_sentences.append({sentence:transformed_sentence})

for data in transformed_sentences:
    for k,v in data.items():
        print(k)
        print(v)
        print()

BACKGROUND	Australia New Zealand Clinical Trials Registry : ACTRN12613000290796 Registered 14 March 2013 .
BACKGROUND Australia New Zealand Clinical Trials Registry : ACTRN12613000290796 Registered fourteen March two thousand and thirteen .

RESULTS	Significant improvements in glucose were observed with repaglinide ( HBA ( 1c ) : -1.5 % , fasting glucose : -2.8 mmol/L , 2-h glucose : -3.7 mmol/L , AUC ( 120 ) : -18.9 % ) and glibenclamide ( -1.0 % , -2.2 mmol/L , -2.5 mmol/L , -17.5 % ) .
RESULTS Significant improvements in glucose were observed with repaglinide ( HBA ( one cents ) : minus one point five % , fasting glucose : minus two point eight mmol/L , two -h glucose : minus three point seven mmol/L , AUC ( one hundred and twenty ) : minus eighteen point nine % ) and glibenclamide ( minus one % , minus two point two mmol/L , minus two point five mmol/L , minus seventeen point five % ) .

RESULTS	Repaglinide was also associated with an increase in the AUC ( 60 ) and AUC ( 120 ) for 

### Print Test Cases in JSON

In [3]:
update_test_cases = True

if update_test_cases:
    test_json_dict = {"type": "numeric_to_word",
                      "test_cases":[]}

    for i in range(len(transformed_sentences)):
        test_json_dict["test_cases"].append({"class": "NumericToWord",
                                             "inputs" : {"sentence":list(transformed_sentences[i].items())[0][0]},
                                             "outputs" : {"sentence":list(transformed_sentences[i].items())[0][1]}}
                                           )

    with open("test.json", "w") as outfile: 
        json.dump(test_json_dict, outfile)